In [1]:
import phoenix as px
session = px.launch_app()
# will provide the URL, which provides the front 
# end for the collector

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/phoenix/trace/dsl/query.py:746: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_attributes = pd.DataFrame.from_records(


In [2]:
# will instrumment the OPENAI
from phoenix.trace.openai import OpenAIInstrumentor

In [3]:
# Auto-instrumentation of OpenAI
OpenAIInstrumentor().instrument()

In [4]:
import os
from openai import OpenAI

In [5]:
from dotenv import load_dotenv
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

True

In [6]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [7]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, can you help me with something?"}
]

In [9]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=conversation,
)

In [10]:
# exporting traces
df = px.Client().get_spans_dataframe()
df.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.input.mime_type,attributes.openinference.span.kind,attributes.llm.input_messages,attributes.llm.model_name,attributes.llm.output_messages,attributes.output.mime_type,attributes.llm.token_count.completion,attributes.llm.token_count.prompt,attributes.output.value,attributes.llm.token_count.total
context.span_id,,,,,,,,,,,,,,,,,,,,,
611691d71b3f5091,ChatCompletion,LLM,None,2024-08-15 15:52:39.076031+00:00,2024-08-15 15:52:39.759912+00:00,ERROR,NotFoundError: Error code: 404 - {'error': {'m...,"[{'attributes': {'exception.escaped': 'False',...",611691d71b3f5091,3ec6da5d8ef31ece0cbccb3ad27e66b9,...,application/json,LLM,[{'message.content': 'You are a helpful assist...,None,None,None,NaN,NaN,None,NaN
d8c24f83b7e48428,ChatCompletion,LLM,None,2024-08-15 15:53:17.648469+00:00,2024-08-15 15:53:18.872164+00:00,OK,,[],d8c24f83b7e48428,43fb7a72b6f0411ee22c3c389aa4f275,...,application/json,LLM,[{'message.content': 'You are a helpful assist...,gpt-4o-mini-2024-07-18,[{'message.content': 'Of course! How can I ass...,application/json,10.0,26.0,"{""id"":""chatcmpl-9wX4oPFyixLXS15IWBfs0AM30oZWz""...",36.0


In [11]:
# LLM evaluation is best done with Human Labelling which doesn't 
# scale. Using LLMs itself to Eval is becoming a possibility. 
# https://docs.arize.com/phoenix/evaluation/llm-evals

In [1]:
# export data from the traces thot you already have

from urllib.request import urlopen
from phoenix.trace.trace_dataset import TraceDataset
from phoenix.trace.utils import json_lines_to_df

# Replace with the URL to your trace data
traces_url = "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/trace.jsonl"

with urlopen(traces_url) as response:
    lines = [line.decode("utf-8") for line in response.readlines()]

trace_ds = TraceDataset(json_lines_to_df(lines))

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
trace_ds.dataframe

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,conversation,context.trace_id,...,attributes.llm.invocation_parameters,attributes.llm.output_messages,attributes.llm.token_count.prompt,attributes.llm.token_count.completion,attributes.llm.token_count.total,attributes.llm.prompt_template.template,attributes.llm.prompt_template.variables,attributes.retrieval.documents,attributes.embedding.model_name,attributes.embedding.embeddings
0,query,CHAIN,None,2023-12-11 17:57:17.891021+00:00,2023-12-11 17:57:20.075141+00:00,OK,,[],None,f40dc5d5-08b7-4e23-80e1-2cd6e9f0cf29,...,None,None,None,None,None,None,None,None,None,None
1,synthesize,CHAIN,bce5b9ae-4587-4ead-9ccc-de3fe29257bc,2023-12-11 17:57:18.973513+00:00,2023-12-11 17:57:20.075056+00:00,OK,,[],None,f40dc5d5-08b7-4e23-80e1-2cd6e9f0cf29,...,None,None,None,None,None,None,None,None,None,None
2,llm,LLM,3d59ca9b-5d68-4773-856f-5243cba51647,2023-12-11 17:57:18.985506+00:00,2023-12-11 17:57:20.074314+00:00,OK,,[],None,f40dc5d5-08b7-4e23-80e1-2cd6e9f0cf29,...,"{""model"": ""gpt-3.5-turbo"", ""temperature"": 0.0,...","[{'message.role': 'assistant', 'message.conten...",385.0,21.0,406.0,system: You are an expert Q&A system that is t...,{'context_str': 'Here I'm using a monitor that...,None,None,None
3,retrieve,RETRIEVER,bce5b9ae-4587-4ead-9ccc-de3fe29257bc,2023-12-11 17:57:17.891487+00:00,2023-12-11 17:57:18.973316+00:00,OK,,[],None,f40dc5d5-08b7-4e23-80e1-2cd6e9f0cf29,...,None,None,None,None,None,None,None,[{'document.id': 'b083ee22-965f-4086-856f-4f66...,None,None
4,embedding,EMBEDDING,eef727de-9f27-4b41-aa79-acaccdf92383,2023-12-11 17:57:17.891757+00:00,2023-12-11 17:57:18.390893+00:00,OK,,[],None,f40dc5d5-08b7-4e23-80e1-2cd6e9f0cf29,...,None,None,None,None,None,None,None,None,text-embedding-ada-002,[{'embedding.text': 'How can I query for a mon...
5,query,CHAIN,None,2023-12-11 17:57:20.101738+00:00,2023-12-11 17:57:22.918341+00:00,OK,,[],None,1bceff06-0bb9-48d6-a498-ca5bc7afbb7d,...,None,None,None,None,None,None,None,None,None,None
6,synthesize,CHAIN,f31c335f-9b7f-4ed7-96eb-ace33a4df064,2023-12-11 17:57:20.612452+00:00,2023-12-11 17:57:22.918299+00:00,OK,,[],None,1bceff06-0bb9-48d6-a498-ca5bc7afbb7d,...,None,None,None,None,None,None,None,None,None,None
7,llm,LLM,fd225452-2243-443d-8553-0973f6420dea,2023-12-11 17:57:20.616799+00:00,2023-12-11 17:57:22.918144+00:00,OK,,[],None,1bceff06-0bb9-48d6-a498-ca5bc7afbb7d,...,"{""model"": ""gpt-3.5-turbo"", ""temperature"": 0.0,...","[{'message.role': 'assistant', 'message.conten...",254.0,65.0,319.0,system: You are an expert Q&A system that is t...,{'context_str': 'Navigate to the custom metric...,None,None,None
8,retrieve,RETRIEVER,f31c335f-9b7f-4ed7-96eb-ace33a4df064,2023-12-11 17:57:20.101886+00:00,2023-12-11 17:57:20.612259+00:00,OK,,[],None,1bceff06-0bb9-48d6-a498-ca5bc7afbb7d,...,None,None,None,None,None,None,None,[{'document.id': 'cb50530e-166e-4045-b76d-850d...,None,None
9,embedding,EMBEDDING,a71236bc-6f1f-46c6-b799-4163048c8c51,2023-12-11 17:57:20.103012+00:00,2023-12-11 17:57:20.242200+00:00,OK,,[],None,1bceff06-0bb9-48d6-a498-ca5bc7afbb7d,...,None,None,None,None,None,None,None,None,text-embedding-ada-002,[{'embedding.text': 'How do I delete a model?'...


In [16]:
# px.close_app()

In [2]:
import phoenix as px
session = px.launch_app(trace=trace_ds)
session.view()

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


In [6]:
from phoenix.evals import OpenAIModel, HallucinationEvaluator, QAEvaluator
from phoenix.evals import run_evals
import nest_asyncio
nest_asyncio.apply()  # This is needed for concurrency in notebook environments

In [7]:
# Set your OpenAI API key
import os
from dotenv import load_dotenv
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")
api_key = "your-api-key"  # Replace with your actual API key
eval_model = OpenAIModel(model="gpt-4o-mini",
                         api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
# Define your evaluators
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_evaluator = QAEvaluator(eval_model)

In [ ]:
# Assume 'queries_df' is your input dataframe 
# for `hallucination_evaluator` your input df needs to have columns 'output', 'input', 'context'
# for `qa_evaluator` your input df needs to have columns 'output', 'input', 'reference'
assert all(column in queries_df.columns for column in ['output', 'input', 'context', 'reference'])

# Run the evaluators, each evaluator will return a dataframe with evaluation results
# We upload the evaluation results to Phoenix in the next step
hallucination_eval_df, qa_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_evaluator],
    provide_explanation=True
# Log the evaluations
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_eval_df)
))